# Fitting for $e$ and $w$

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
import scipy.constants as c
from tqdm import tqdm
import PyAstronomy.pyasl as pya

import photoeccentric as ph
import emcee
import corner

np.set_printoptions(threshold=sys.maxsize)
plt.rcParams['figure.figsize'] = [10, 5]

In [2]:
def get_T23(p, rp_earth, rs, T14, a, i):
    
    ing_eg = 2*pya.ingressDuration(a, rp_earth*11.2, rs, i, p)#rp needs to in jovian radii
    T23 = T14-ing_eg
    
    return T23

def get_T23_errs(T23_dist):
    
    x, cdf = mt.get_cdf(T23_dist)
    T23_sigma_minus = mt.find_sigma(x, cdf, "-")
    T23_sigma_plus = mt.find_sigma(x, cdf, "+")
    
    return T23_sigma_minus, T23_sigma_plus


def get_planet_params(p, T14, T23):
    """Returns planet parameters in correct units.

    Parameters
    ----------
    p: float
        Planet orbital period (days)
    rp_earth: float
        Planet radius (earth radii)
    rs: float
        Stellar radius (solar radii)
    T14: float
        Total transit time - first to fourth contact (hours)
    a: float
        Planet semi-major axis (AU)
    i: float
        Orbital inclination (degrees)

    Returns
    -------
    p_seconds: float
        Orbital period (seconds)
    rprs: float
        Planet radius (stellar host radii)
    T14_seconds: float
        Total transit time - first to fourth contact (seconds)
    T23_seconds: float
        Full transit time - second to third contact (seconds)
    """

    p_seconds = p*86400
    T14_seconds = T14*3600
    T23_seconds = T23*3600

    return p_seconds, T14_seconds, T23_seconds


def get_rho_circ(rprs, T14, T23, p):
    """Returns stellar density, assuming a perfectly circular planetary orbit.

    Parameters
    ----------
    rprs: float
        Planet radius (stellar host radii)
    T14: float
        Total transit time - first to fourth contact (seconds)
    T23: float
        Full transit time - second to third contact (seconds)
    p: float
        Orbital period (seconds)

    Returns
    -------
    rho_circ: float
        Stellar density, assuming a circular orbit (kg/m^3)
    """
    
    delta = rprs**2
    num1 = 2*(delta**(0.25))
    den1 = np.sqrt((T14**2)-(T23**2))
    term1 = (num1/den1)**3

    num2 = 3*p
    den2 = c.G*(c.pi**2)
    term2 = num2/den2

    rho_circ = term1*term2
    
    return rho_circ


def get_g(rho_circ, rho_star):
    """Gets g

    Parameters
    ----------
    rho_circ: float
        Stellar density, assuming a circular orbit (kg/m^3)
    rho_star: float
        Stellar density, calculated from Kepler/Gaia/spectroscopy (kg/m^3)

    Returns
    -------
    g: float
        Cube root of ratio between rho_circ and rho_star
    """
    g = np.cbrt(rho_circ/rho_star)
    return g

def get_g_from_def(e, w):
    """Gets g from e and omega

    Parameters
    ----------
    e: float
        Eccentricity
    w: float
        Angle of periapse or something

    Returns
    -------
    g: float
        Cube root of ratio between rho_circ and rho_star
    """ 
    g = (1+e*np.sin(w*(np.pi/180.)))/np.sqrt(1-e**2)
    return g


def get_e(g, w):
    """Gets eccentricity (from photoeccentric effect)

    Parameters
    ----------
    g: float
        Cube root of ratio between rho_circ and rho_star
    w: float
        Angle of apoapse or periapse (?) (degrees, -90 < w < 90)

    Returns
    -------
    e: float
        Eccentricity of planet orbit
    """
    e = (np.sqrt(2)*(np.sqrt(2*g**4 - g**2*np.cos(2*w) - g**2 - 2*np.sin(w*(np.pi/180.)))))/(2*(g**2 + np.sin(w*(np.pi/180.)**2)))
    return e

1. Read in rhos for all Kepler stars with planets (with spectroscopy)
2. Read in all Kepler stars with planets (from exoplanet archive)
3. Take data from (2) for stars in (1)

In [3]:
#Kepler planets with spectroscopy
rho_lum = pd.read_csv('/Users/sheilasagear/Dropbox (UFL)/Research/MetallicityProject/Data/rhos/rho_lum.csv')

# ALL Kepler planets from exo archive
planets = pd.read_csv('/Users/sheilasagear/Dropbox (UFL)/Research/MetallicityProject/Data/exoplanetarchive/cumulative_kois.csv')

spectplanets = planets[planets['kepid'].isin(list(rho_lum.columns))]
spectplanets = spectplanets.reset_index()

In [4]:
spectplanets = spectplanets.reset_index()

In [5]:
def row_to_top(df, index):
    """Bring row to top
    
    Parameters
    ----------
    df: pandas.dataframe
        Dataframe to copy
    index: int
        Index of row to bring to top
        
    Returns
    -------
    df_cp: pandas.dataframe
        Copy of dataframe with specified row at top
    
    """
    df_cp = pd.concat([df.iloc[[index],:], df.drop(index, axis=0)], axis=0)
    return df_cp

In [6]:

def get_g_distribution(row, n_rhos):
    """Gets g distribution for a KOI.
    
    Parameters
    ----------
    row: int
        Row in pandas.dataframe of info from Exoplanet Archive. (change this to take KIC/KOI)
    n_rhos: int
        Number of values in distribution
        
    Returns
    -------
    gs: np.array
        g distribution for star/planet.
    """

    targ = spectplanets.iloc[row]
    print('KIC: ', targ.kepid)

    rhos = rho_lum[str(targ.kepid)].dropna()
    rhos = np.array(rhos)

    while len(rhos) > n_rhos:
        rhos = np.delete(rhos, [np.random.randint(0, len(rhos))])

    #ws = np.arange(-90., 300., 1.)

    gs = np.zeros((len(rhos)))
    #es = np.zeros(len(rhos))
    #es = np.zeros((len(ws), len(rhos)))

    rho_circ = np.zeros(len(rhos))
    rho_ratios = np.zeros(len(rhos))
    T23_dist = np.zeros((len(rhos)))

    per_dist = mt.asymmetric_gaussian(targ.koi_period, targ.koi_period_err1, targ.koi_period_err2, len(rhos))
    
    rs_dist = mt.asymmetric_gaussian(targ.koi_srad, targ.koi_srad_err1, targ.koi_srad_err2, len(rhos))
    rp_earth_dist = mt.asymmetric_gaussian(targ.koi_prad, targ.koi_prad_err1, targ.koi_prad_err2, len(rhos))
    rprs_dist = mt.asymmetric_gaussian(targ.koi_ror, targ.koi_ror_err1, targ.koi_ror_err2, len(rhos))

    T14_dist = mt.asymmetric_gaussian(targ.koi_duration, targ.koi_duration_err1, targ.koi_duration_err2, len(rhos))

    a = targ.koi_sma
    i = targ.koi_incl

    for j in tqdm(range(len(rhos))): #for element in histogram for star:
        T23_dist[j] = get_T23(per_dist[j], rp_earth_dist[j], rs_dist[j], T14_dist[j], a, i)
        
        p_seconds, T14_seconds, T23_seconds = get_planet_params(per_dist[j], T14_dist[j], T23_dist[j])
        rho_circ[j] = get_rho_circ(rprs_dist[j], T14_seconds, T23_seconds, p_seconds)

        rho_ratios[j] = rho_circ[j]/rhos[j]
        g = get_g(rho_circ[j], rhos[j])
        gs[j] = g

    return gs
    

In [7]:
spectplanets

,level_0,index,kepid,kepoi_name,kepler_name,koi_disposition,koi_vet_stat,koi_vet_date,koi_pdisposition,koi_score,...,koi_dicco_mdec,koi_dicco_mdec_err,koi_dicco_msky,koi_dicco_msky_err,koi_dikco_mra,koi_dikco_mra_err,koi_dikco_mdec,koi_dikco_mdec_err,koi_dikco_msky,koi_dikco_msky_err
0,0,64,11923270,K00781.01,Kepler-676 b,CONFIRMED,Done,8/16/18,CANDIDATE,1.000,...,-0.03,0.082,0.066,0.095,0.29,0.110,0.628,0.080,0.690,0.086
1,1,85,4139816,K00812.02,Kepler-235 d,CONFIRMED,Done,8/16/18,CANDIDATE,1.000,...,-0.03,0.140,0.030,0.140,-0.18,0.140,-0.150,0.140,0.230,0.140
2,2,86,4139816,K00812.03,Kepler-235 e,CONFIRMED,Done,8/16/18,CANDIDATE,1.000,...,0.04,0.220,0.080,0.210,-0.25,0.170,-0.060,0.160,0.260,0.170
3,3,88,4139816,K00812.04,Kepler-235 c,CONFIRMED,Done,8/16/18,CANDIDATE,0.994,...,-0.45,0.390,0.550,0.350,0.11,0.300,-0.580,0.430,0.590,0.400
4,4,105,4139816,K00812.01,Kepler-235 b,CONFIRMED,Done,8/16/18,CANDIDATE,1.000,...,0.05,0.097,0.060,0.110,-0.15,0.092,-0.039,0.092,0.154,0.092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,92,7186,5384713,K03444.04,NaN,CANDIDATE,Done,8/16/18,CANDIDATE,0.000,...,1.20,0.720,1.210,0.730,0.21,0.930,1.570,0.890,1.580,0.910
93,93,7588,10122538,K02926.03,Kepler-1388 d,CONFIRMED,Done,8/16/18,CANDIDATE,1.000,...,-0.06,0.190,0.140,0.210,-0.10,0.100,-0.180,0.230,0.210,0.210
94,94,7606,10122538,K02926.04,Kepler-1388 e,CONFIRMED,Done,8/16/18,CANDIDATE,1.000,...,0.15,0.220,0.250,0.220,-0.02,0.220,0.000,0.200,0.020,0.220
95,95,8590,11129738,K01427.02,NaN,CANDIDATE,Done,8/16/18,CANDIDATE,0.891,...,1.05,0.460,1.140,0.460,-0.35,0.460,0.810,0.460,0.880,0.460


In [8]:
gs_11923270 = get_g_distribution(0, 500)
gs_4139816d = get_g_distribution(1, 500)
gs_6435936 = get_g_distribution(5, 500)

KIC:  11923270


100%|██████████| 500/500 [01:23<00:00,  6.00it/s]


KIC:  4139816


100%|██████████| 500/500 [01:20<00:00,  6.22it/s]


KIC:  6435936


100%|██████████| 500/500 [01:08<00:00,  7.32it/s]


In [9]:
gs_7907423 = get_g_distribution(6, 500)

KIC:  7907423


 75%|███████▌  | 375/500 [00:50<00:16,  7.41it/s]


KeyboardInterrupt: 

In [ ]:
gs_4139816e = get_g_distribution(2, 200)
gs_4139816c = get_g_distribution(3, 200)
gs_4139816v = get_g_distribution(4, 200)

In [ ]:
def get_sigmas(dist):
    """Gets + and - sigmas from a distribution (gaussian or not) through a cdf
    
    Parameters
    ----------
    dist: np.array
        Distribution from which sigmas are needed
        
    Returns
    -------
    sigma_minus: float
        - sigma
    sigma_plus: float
        + sigma
    """
    x, cdf = mt.get_cdf(dist)
    sigma_minus = mt.find_sigma(x, cdf, "-")
    sigma_plus = mt.find_sigma(x, cdf, "+")
    
    return sigma_minus, sigma_plus

In [ ]:
sigma_m, sigma_p = get_sigmas(gs_11923270)

sigma_m2, sigma_p2 = get_sigmas(rho_lum['11923270'].dropna())

In [ ]:
plt.hist(rho_lum['11923270'].dropna());
plt.xlabel('rho (kg m^-3)');
plt.title('KIC 11923270');
plt.axvline(np.mean(rho_lum['11923270'].dropna())+sigma_m2, c='r')

In [ ]:
plt.hist(gs_11923270, bins='auto');
plt.xlabel('g', fontsize=20)
plt.title('Distribution of $g$s for KIC 11923270', fontsize=20)
plt.axvline(np.mean(gs_11923270)+sigma_m, c='r')
plt.axvline(np.mean(gs_11923270)+sigma_p, c='r')

In [ ]:
def get_e_from_def(g, w):
    """Gets eccentricity from definition (eqn 4, not really the definition tho)
    
    Parameters
    ----------
    g: float
        g value
    w: float
        Omega (angle periapse/apoapse)
    
    Returns
    -------
    e: float
        Eccentricity calculated solely on g and w
    
    """
    num = np.sqrt(2)*np.sqrt(2*g**4-g**2*np.cos(2*w)-g**2-2*np.sin(w*(np.pi/180.)))
    den = 2*(g**2+np.sin(w*(np.pi/180.)**2))
    e = num/den
    return e

## Finding the allowable (e, w) space for one planet with MCMC

In [ ]:
def log_likelihood(theta, g, gerr):
    """Log of likelihood
    model = g(e,w)
    gerr = sigma of g distribution
    """
    w, e = theta
    model = (1+e*np.sin(w*(np.pi/180.)))/np.sqrt(1-e**2)
    sigma2 = gerr ** 2
    return -0.5 * np.sum((g - model) ** 2 / sigma2 + np.log(sigma2))

In [ ]:
def log_prior(theta):
    """Log of prior
    e between 0 and 1
    w between -90 and 300
    """
    w, e = theta
    if 0.0 < e < 1.0 and -90.0 < w < 300.0:
        return 0.0
    return -np.inf

In [ ]:
def log_probability(theta, g, gerr):
    """Log of probability
    """
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, g, gerr)

## For planet with g = 1.1

In [ ]:
#Guesses
w_guess = 100.0
e_guess = 0.95

solnx = (w_guess, e_guess)

In [ ]:
pos = solnx + 1e-4 * np.random.randn(32, 2)
nwalkers, ndim = pos.shape

In [ ]:
g_mean = np.mean(gs_11923270)
g_sigma = np.mean(np.abs(get_sigmas(gs_11923270)))

#g_mean = 0.4
#g_sigma = 0.05

In [ ]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=(g_mean, g_sigma), threads=4)
sampler.run_mcmc(pos, 7000, progress=True);

In [ ]:
fig, axes = plt.subplots(2, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()
labels = ["w", "e"]
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)
    ax.set_xlim(0, 500)

axes[-1].set_xlabel("step number");

In [ ]:
flat_samples = sampler.get_chain(discard=100, thin=15, flat=True)
fig = corner.corner(flat_samples, labels=labels)

## For planet with g = 0.85

In [ ]:
#Guesses
w_guess = 0.0
e_guess = 0.40

solnx = (w_guess, e_guess)

In [ ]:
pos = solnx + 1e-4 * np.random.randn(64, 2)
nwalkers, ndim = pos.shape

In [ ]:
g_mean = np.mean(gs_6435936)
g_sigma = np.mean(np.abs(get_sigmas(gs_6435936)))

In [ ]:
g_mean

In [ ]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=(g_mean, g_sigma), threads=4)
sampler.run_mcmc(pos, 10000, progress=True);

In [ ]:
fig, axes = plt.subplots(2, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()
labels = ["w", "e"]
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)
    ax.set_xlim(0, 500)

axes[-1].set_xlabel("step number");

In [ ]:
flat_samples = sampler.get_chain(discard=100, thin=15, flat=True)
fig = corner.corner(flat_samples, labels=labels)

## For planet with g = 0.75

In [ ]:
#Guesses
w_guess = 100.0
e_guess = 0.55

solnx = (w_guess, e_guess)

In [ ]:
pos = solnx + 1e-4 * np.random.randn(32, 2)
nwalkers, ndim = pos.shape

In [ ]:
g_mean = np.mean(gs_7907423)
g_sigma = np.mean(np.abs(get_sigmas(gs_7907423)))

In [ ]:
g_mean

In [ ]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=(g_mean, g_sigma), threads=4)
sampler.run_mcmc(pos, 10000, progress=True);

In [ ]:
fig, axes = plt.subplots(2, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()
labels = ["w", "e"]
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)
    ax.set_xlim(0, 500)

axes[-1].set_xlabel("step number");

In [ ]:
flat_samples = sampler.get_chain(discard=100, thin=15, flat=True)
fig = corner.corner(flat_samples, labels=labels)

# A 4-planet system: 4139816

## 4139816b

g=0.9

In [ ]:
gs_4139816b = gs_4139816v

In [ ]:
#Guesses
w_guess = 100.0
e_guess = 0.55

solnx = (w_guess, e_guess)

In [ ]:
pos = solnx + 1e-4 * np.random.randn(32, 2)
nwalkers, ndim = pos.shape

In [ ]:
g_mean = np.mean(gs_4139816b)
g_sigma = np.mean(np.abs(get_sigmas(gs_4139816b)))

In [ ]:
g_mean

In [ ]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=(g_mean, g_sigma), threads=4)
sampler.run_mcmc(pos, 8000, progress=True);

In [ ]:
fig, axes = plt.subplots(2, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()
labels = ["w", "e"]
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)
    ax.set_xlim(0, 500)

axes[-1].set_xlabel("step number");

In [ ]:
flat_samples = sampler.get_chain(discard=100, thin=15, flat=True)
fig = corner.corner(flat_samples, labels=labels)

## 4139816c

g=0.73

In [ ]:
#Guesses
w_guess = 100.0
e_guess = 0.55

solnx = (w_guess, e_guess)

In [ ]:
pos = solnx + 1e-4 * np.random.randn(32, 2)
nwalkers, ndim = pos.shape

In [ ]:
g_mean = np.mean(gs_4139816c)
g_sigma = np.mean(np.abs(get_sigmas(gs_4139816c)))

In [ ]:
g_mean

In [ ]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=(g_mean, g_sigma), threads=4)
sampler.run_mcmc(pos, 8000, progress=True);

In [ ]:
fig, axes = plt.subplots(2, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()
labels = ["w", "e"]
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)
    ax.set_xlim(0, 500)

axes[-1].set_xlabel("step number");

In [ ]:
flat_samples = sampler.get_chain(discard=100, thin=15, flat=True)
fig = corner.corner(flat_samples, labels=labels)

## 4139816d
g = 0.9

In [ ]:
#Guesses
w_guess = 100.0
e_guess = 0.55

solnx = (w_guess, e_guess)

In [ ]:
pos = solnx + 1e-4 * np.random.randn(32, 2)
nwalkers, ndim = pos.shape

In [ ]:
g_mean = np.mean(gs_4139816d)
g_sigma = np.mean(np.abs(get_sigmas(gs_4139816d)))

In [ ]:
g_mean

In [ ]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=(g_mean, g_sigma), threads=4)
sampler.run_mcmc(pos, 8000, progress=True);

In [ ]:
fig, axes = plt.subplots(2, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()
labels = ["w", "e"]
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)
    ax.set_xlim(0, 500)

axes[-1].set_xlabel("step number");

In [ ]:
flat_samples = sampler.get_chain(discard=100, thin=15, flat=True)
fig = corner.corner(flat_samples, labels=labels)

## 4139816e
g=0.86

In [ ]:
#Guesses
w_guess = 50.0
e_guess = 0.3

solnx = (w_guess, e_guess)

In [ ]:
pos = solnx + 1e-4 * np.random.randn(32, 2)
nwalkers, ndim = pos.shape

In [ ]:
g_mean = np.mean(gs_4139816e)
g_sigma = np.mean(np.abs(get_sigmas(gs_4139816e)))

In [ ]:
g_mean

In [ ]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=(g_mean, g_sigma), threads=4)
sampler.run_mcmc(pos, 10000, progress=True);

In [ ]:
fig, axes = plt.subplots(2, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()
labels = ["w", "e"]
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)
    ax.set_xlim(0, 500)

axes[-1].set_xlabel("step number");

In [ ]:
flat_samples = sampler.get_chain(discard=100, thin=15, flat=True)
fig = corner.corner(flat_samples, labels=labels)

#### Graph of g from (e, w) meshgrid

In [ ]:
e = np.linspace(0.0, 1.0, num=500)
w = np.linspace(-90.0, 300.0, num=500)
W, E = np.meshgrid(w, e)

In [ ]:
G = get_g_from_def(E, W)

In [ ]:
from matplotlib import ticker, cm

In [ ]:
plt.contourf(W, E, G, levels=200, locator=ticker.LogLocator(), cmap='Blues')
plt.colorbar()

In [ ]:
fig, ax = plt.subplots()
CS = ax.contourf(W, E, G, levels=100)
ax.set_ylabel('e');
ax.set_xlabel('w (degrees)')

In [ ]:
for i in range(len(G)):
    for j in range(len(G[0])):
        if 0.7 < G[i][j] < 1.3:
            G[i][j] = 1.
        else:
            G[i][j] = 0.

In [ ]:
fig, ax = plt.subplots()
CS = ax.contourf(W, E, G)
#ax.clabel(CS, inline=True, fontsize=10)
ax.set_ylabel('e');
ax.set_xlabel('w (degrees)')
ax.set_title('g = 1.0 +/- 0.2');

😈

Create a transit with e, see if you can get it back out with above code ^

https://www.cfa.harvard.edu/~lkreidberg/batman/tutorial.html

I'll assume an M5 with mass 0.14 +/- 0.02 Msol and radius 0.2 +/- 0.01 Rsol

In [20]:
masses = np.random.normal(0.14, 0.02, size=1000)
masserr = np.random.normal(0.0, 0.02, size=1000)
radii = np.random.normal(0.2, 0.01, size=1000)
raderr = np.random.normal(0.0, 0.01, size=1000)

In [23]:
ph.find_density_dist_symmetric(1, masses, masserr, radii, raderr)

  0%|          | 0/1 [00:00<?, ?it/s]


NameError: name 'masserr2' is not defined

In [ ]:
import batman

In [ ]:
params = batman.TransitParams()       #object to store transit parameters
params.t0 = 0.                        #time of inferior conjunction
params.per = 0.1                      #orbital period
params.rp = 0.05                       #planet radius (in units of stellar radii)
params.a = 15.                        #semi-major axis (in units of stellar radii)
params.inc = 87.                      #orbital inclination (in degrees)
params.ecc = 0.5                       #eccentricity
params.w = 50.                        #longitude of periastron (in degrees)
params.limb_dark = "nonlinear"        #limb darkening model
params.u = [0.5, 0.1, 0.1, -0.1]      #limb darkening coefficients [u1, u2, u3, u4]

t = np.linspace(-0.025, 0.025, 1000)  #times at which to calculate light curve
m = batman.TransitModel(params, t)    #initializes model

In [ ]:
flux = m.light_curve(params)                    #calculates light curve

In [ ]:
plt.plot(t, flux)

In [ ]:
time = np.linspace(-12, 12, 4000)

In [ ]:
def planetlc(time, per, rp, a, inc):
    params = batman.TransitParams()       #object to store transit parameters
    params.t0 = 00.                        #time of inferior conjunction
    params.per = per                      #orbital period
    params.rp = rp                       #planet radius (in units of stellar radii)
    params.a = a                        #semi-major axis (in units of stellar radii)
    params.inc = inc                      #orbital inclination (in degrees)
    params.ecc = 0.4                       #eccentricity
    params.w = 100.                        #longitude of periastron (in degrees)
    params.limb_dark = "nonlinear"        #limb darkening model
    params.u = [0.5, 0.1, 0.1, -0.1]      #limb darkening coefficients [u1, u2, u3, u4]
    
    #times at which to calculate light curve
    m = batman.TransitModel(params, time)
    
    flux = m.light_curve(params)
    
    return flux

In [ ]:
flux = planetlc(time, 5, 0.07, 27, 87)

In [ ]:
# Adding gaussian noise
noise = np.random.normal(0,0.0001,len(time))
nflux = flux+noise

In [ ]:
#plt.plot(time, flux)
plt.plot(time, flux+noise)

In [ ]:
import astropy.units as u
from astropy.timeseries import BoxLeastSquares

In [ ]:
mod = BoxLeastSquares(time*u.day, flux, dy=0.01)
periodogram = mod.autopower(0.2, objective="snr")

In [ ]:
plt.plot(periodogram.period, periodogram.power)
periodogram.period = np.asarray(periodogram.period)
per_guess = periodogram.period[int(np.median(np.argmax(periodogram.power)))]

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
p0=[per_guess, 0.08, 30, 88]

In [ ]:
popt, pcov = curve_fit(planetlc, xdata=time, ydata=flux, p0=p0)

In [ ]:
fit = planetlc(time, *popt)

In [ ]:
plt.plot(time, flux)
plt.plot(time, fit)

In [ ]:
p = popt[0]
rprs = popt[1]
# Assuming 0.5 Rsun stellar radius
a = popt[2]*.005*0.5
i = popt[3]

In [ ]:

def get_g_distribution(row, n_rhos):
    """Gets g distribution for a KOI.
    
    Parameters
    ----------
    row: int
        Row in pandas.dataframe of info from Exoplanet Archive. (change this to take KIC/KOI)
    n_rhos: int
        Number of values in distribution
        
    Returns
    -------
    gs: np.array
        g distribution for star/planet.
    """

    targ = spectplanets.iloc[row]
    print('KIC: ', targ.kepid)

    rhos = rho_lum[str(targ.kepid)].dropna()
    rhos = np.array(rhos)

    while len(rhos) > n_rhos:
        rhos = np.delete(rhos, [np.random.randint(0, len(rhos))])

    #ws = np.arange(-90., 300., 1.)

    gs = np.zeros((len(rhos)))
    #es = np.zeros(len(rhos))
    #es = np.zeros((len(ws), len(rhos)))

    rho_circ = np.zeros(len(rhos))
    rho_ratios = np.zeros(len(rhos))
    T23_dist = np.zeros((len(rhos)))

    per_dist = mt.asymmetric_gaussian(targ.koi_period, targ.koi_period_err1, targ.koi_period_err2, len(rhos))
    
    rs_dist = mt.asymmetric_gaussian(targ.koi_srad, targ.koi_srad_err1, targ.koi_srad_err2, len(rhos))
    rp_earth_dist = mt.asymmetric_gaussian(targ.koi_prad, targ.koi_prad_err1, targ.koi_prad_err2, len(rhos))
    rprs_dist = mt.asymmetric_gaussian(targ.koi_ror, targ.koi_ror_err1, targ.koi_ror_err2, len(rhos))

    T14_dist = mt.asymmetric_gaussian(targ.koi_duration, targ.koi_duration_err1, targ.koi_duration_err2, len(rhos))

    a = targ.koi_sma
    i = targ.koi_incl

    for j in tqdm(range(len(rhos))): #for element in histogram for star:
        T23_dist[j] = get_T23(per_dist[j], rp_earth_dist[j], rs_dist[j], T14_dist[j], a, i)
        
        p_seconds, T14_seconds, T23_seconds = get_planet_params(per_dist[j], T14_dist[j], T23_dist[j])
        rho_circ[j] = get_rho_circ(rprs_dist[j], T14_seconds, T23_seconds, p_seconds)

        rho_ratios[j] = rho_circ[j]/rhos[j]
        g = get_g(rho_circ[j], rhos[j])
        gs[j] = g

    return gs
    
    